# Entrega 1 - Introducción al Procesamiento del Lenguaje Natural 2018 


Verifique que su entorno de Python 3 contiene todas las bibliotecas necesarias. Importe las bibliotecas nltk y sklearn y verifique que se importan sin errores.

In [ ]:
import nltk
nltk.download('punkt')
import sklearn
import json
import os
import re
from pprint import pprint
from random import shuffle, seed

## Lectura de datos

Observe el corpus contenido en el directorio *restaurante-review-dataset* extraído de [1]. Cargue el contenido del corpus en una variable de nombre *corpus*. Utilice como estructura de datos una lista de pares (comentario, valor), donde comentario es una string con el comentario y valor corresponde a la string "POS" o "NEG" si el comentario es positivo o negativo, respectivamente. Despliegue la cantidad de comentarios positivos y negativos.
 
[1] Dubiau, L., & Ale, J. M. (2013). Análisis de Sentimientos sobre un Corpus en Español: Experimentación con un Caso de Estudio. In Proceedings of the 14th Argentine Symposium on Artificial Intelligence, ASAI (pp. 36-47).


In [ ]:
path = "dataset/restaurante-review-dataset/"
posPath = path + "pos/"
negPath = path + "neg/"
posFiles = os.listdir(posPath)
negFiles = os.listdir(negPath)
posJsons = [json.load(open(posPath + file)) for file in posFiles]
negJsons = [json.load(open(negPath + file)) for file in negFiles]
posCorpus = [(comentario,"POS") for lista in posJsons for comentario in lista]
negCorpus = [(comentario,"NEG") for lista in negJsons for comentario in lista]
corpus = posCorpus + negCorpus
print(str(len(posCorpus)) + " comentarios positivos y " + str(len(negCorpus)) + " comentarios negativos.")

Guarde los comentarios (sin importar el valor que tenga asignado) en una lista llamada *comentarios*. Reordene esta lista aleatoriamente. Utilice una semilla para el generador aleatorio para que sus resultados sean reproducibles. Despliegue los 3 primeros elementos de la lista *comentarios*.

In [ ]:
comentarios = [comentario for (comentario,_) in corpus]
seed(3)
shuffle(comentarios)
pprint(comentarios[0:3])

## Tokenización

Transforme cada elemento de *comentarios* en una lista de palabras utilizando las funciones de tokenización de oraciones de nltk. Almacene el resultado en una variable llamada *corpus_tokens*. Note que *corpus_tokens* es una lista donde cada elemento es a su vez una lista con los *tokens* del comentario correspondiente.

In [ ]:
#Usamos primero sent_tokenize para partir los comentarios en oraciones y luego word_tokenize para partir esas oraciones en palabras

corpus_tokens = [[word for sent in nltk.tokenize.sent_tokenize(comment, language='spanish') for word in nltk.tokenize.word_tokenize(sent)] for comment in comentarios]

Muestre los dos últimos elementos de *corpus_tokens*.

In [ ]:
pprint(corpus_tokens[-2:])

Realice otra tokenización de los *comentarios* utilizando expresiones regulares y el módulo re de Python. En esta oportunidad cumpla con las siguientes restricciones:

- Reconocer tokens de por lo menos 3 caracteres de largo.
- No reconocer símbolos de puntuación.
- No reconocer tokens únicamente numéricos.

Almacene el resultado en una variable llamada *corpus_tokens2*.

In [ ]:
corpus_tokens2 = [re.findall(r'\b([^\W\d]{3,}|\d+[^\W\d]{2,}|[^\W\d]{2,}\d+|\d+[^\W\d]+\d+|\d+[^\W\d]+\d+[^\W\d]*\d+)\b', comment) for comment in comentarios]

Despliegue los dos últimos elementos de *corpus_tokens2*.

In [ ]:
pprint(corpus_tokens2[-2:])

# Bag of Words

Utilice la clase CountVectorizer del módulo sklearn.feature_extraction.text para instanciar un objeto que transforme el corpus en un diccionario de las palabras más representativas de las oraciones del corpus. Investigue cada uno de los parámetros que acepta su constructor y configurelos de forma que considere adecuada. Justifique sus decisiones.

In [ ]:
tokenRegex = '[a-záéíóúñ]{3,}'
transf = sklearn.feature_extraction.text.CountVectorizer(max_df=0.5, min_df=100, ngram_range=(1,1), analyzer='word', 
                                                         lowercase=True, token_pattern=tokenRegex, stop_words=None)



- Se configura el parámetro max_df en 0.5 para no tener en cuenta palabras que aparezcan en más de la mitad de los comentarios, debido a que estas difícilmente sean representativas del mismo.

- El parámetro min_df por su parte se configura en 100 para no tener en cuenta palabras que ocurran en menos de 100 comentarios. Sabiendo que hay más de 50 mil comentarios, si una palabra aparece en menos de 100, probablemente esté mal escrita o sea un término muy específico, por lo cual no ayuda a caracterizar el comentario.

- Para tener en cuenta únicamente palabras (y no caracteres o conjuntos de palabras) se setea el parámetro analyzer en 'word' y ngram_range en (1,1).

- El parámetro lowercase se setea en True para que las mayúsculas y minúsculas sean indiferentes.

- En token_pattern se configura una expresión regular que reconoce palabras de 3 o más letras, incluyendo tildes, pero que evita reconocer por ejemplo números o caracteres especiales.

- Debido al uso de max_df se vuelve innecesario el uso de stop_words. En vez de indicar una lista de palabras a ignorar, se ignoran las palabras que aparecen con demasiada frecuencia.

Entrene la transformación e imprima las palabras resultantes en el diccionario. Imprima además la cantidad de palabras obtenidas en el diccionario.

In [ ]:
transf.fit(comentarios)

In [ ]:
diccionario = transf.get_feature_names()
print(diccionario)
print("El diccionario tiene " + str(len(diccionario)) + " palabras.")

Transforme todos los *comentarios* en vectores utilizando la transformación creada. Guarde las transformaciones en una lista llamada 'vectores':

In [ ]:
vectores = transf.transform(comentarios)

## Similitud

Escriba una función calcular_similitud(vect1, vect2) que dados dos vectores, devuelva un número en el rango [0, 1] donde 1 significa que los vectores son idénticos y 0 que son completamente diferentes. Puede utilizar medidas como la implementada en la función cosine del módulo scipy.spatial.distance u otra que considere adecuada.

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine as dist 

def calcular_similitud(vec1, vec2):
  normaVec1 = np.linalg.norm(vec1)
  normaVec2 = np.linalg.norm(vec2)
  if (dist.__name__ in ["cosine", "braycurtis"]) and (normaVec1 == 0 or normaVec2 == 0):
    return 0
  # Como las implementaciones de las distancias de coseno y Bray-Curtis no admiten vectores nulos, se debe chequear 
  # este aspecto. En caso de que alguno de los dos cumpla esto, se los considera diferentes.
  distancia = dist(vec1, vec2)
  return 1/(1 + distancia) 

Como las funciones de distancia son crecientes y además no garantizan un valor entre 0 y 1 (a excepción de las distancias de coseno y bray-curtis), la función devuelve `1/(1 + distancia(v1, v2))` para obtener el valor de similitud.

Vamos a definir los siguientes dos diccionarios:

In [ ]:
similar = {}
similitud = {}

El primer diccionario guardará para cada oración, el índice de la oración más similar a ella. Esto es, similar[i] = j indica que la j-esima oración, es la oración más similar a la i-esima oración.
El segundo diccionario guardará el valor de similitud correspondiente. Si similiar[i] = j, entonces similitud[i] = calcular_similitud(oracion[i], oracion[j]).
Implemente una fución llamada calcular_similitudes(vectores, similar, similitud) que dada la lista de vectores, construya los diccionarios similar y similitud:

In [ ]:
from heapq import heappush

def calcular_similitudes(vectores, similar, similitud):
  cantVectores = len(vectores)
  similitudes = [[] for _ in range(cantVectores)]
  for i in range(cantVectores):
    for j in range(i+1, cantVectores):
        dist = calcular_similitud(vectores[i], vectores[j])
        heappush(similitudes[i], (dist, j))
        heappush(similitudes[j], (dist, i))
    maximo = max(similitudes[i])
    similitud[i] = maximo[0]
    similar[i] = maximo[1]

En esta función se utiliza una cola de prioridad (modelada como un heap) para almacenar una dupla con los valores de similitud y el comentario al que corresponde dicho valor de manera ordenada. Cuando se terminan de computar las distancias de un vector con el resto de los vectores, se obtiene el valor máximo y se inserta en valor de similitud en `similitud` y el comentario al que corresponde en `similar`.

Implemente una función llamada *imprimir_similares* que imprima los primeros N pares de comentarios más similares del corpus. Utilice un valor que considere adecuado para N (ej. N=50).

In [ ]:
from operator import itemgetter

def imprimir_similares(comentarios, similar, similitud, cantComentarios=50):
  simlitudesEnumeradas = list(zip(similar.keys(), similitud.values()))
  # Para conservar a que comentario corresponde cada valor
  similitudesOrdenadas = sorted(simlitudesEnumeradas, key=itemgetter(1)) 
  # Se ordenan las similitudes
  comentariosImpresos = 0
  i = len(similitud) - 1
  impresos = [False for _ in range(len(similitud))]
  while comentariosImpresos < cantComentarios and i < len(comentarios):
    posComentario = similitudesOrdenadas[i][0]
    posComentarioSimilar = similar[posComentario]
    if not impresos[posComentario] or not impresos[posComentarioSimilar]:
        print("El comentario nro. {}:\n\n{}\n".format(posComentario, comentarios[posComentario])) 
        print("Es similar al nro. {}:\n\n{}\n".format(posComentarioSimilar, comentarios[posComentarioSimilar]))
        print("Con similitud {}\n\n".format(round(similitud[posComentario], 3)))
        print("*************************************\n") 
        impresos[posComentario] = True
        impresos[posComentarioSimilar] = True
    comentariosImpresos += 1
    i -= 1

Para no imprimir el mismo par de comentario dos veces, se guarda en `impresos` los comentarios que ya se imprimieron. Si un par ya fue impreso, no se vuelve a imprimir. Esto puede suceder cuando dos comentarios son similares maximalmente entre ellos.

Ejecute las funciones calcular_similitudes e imprimir_similares. Utilice una porción del corpus que considere adecuada para que ejecute en un tiempo razonable (ej. los primeros 1000 comentarios). Despliegue los resultados y el tiempo transcurrido. Si lo desea puede incluir resultados en un bloque de texto al considerar una porción mayor. De ser así incluya la cantidad de elementos considerados y el tiempo transcurrido. 

In [ ]:
from timeit import default_timer as timer

tComienzo = timer()
calcular_similitudes(vectores.toarray()[:1000], similar, similitud)
imprimir_similares(comentarios, similar, similitud)
tFinal = timer()
print("Demoró {} segundos.".format(round(tFinal - tComienzo, 3)))

En las pruebas realizadas anteriormente, se transforma la matriz dispersa en un arreglo de vectores. A continuación, mostramos los resultados utilizando operaciones para trabajar con matrices dispersas.

Primero, redefinimos las funciones calcular_similitud y calcular_similitudes para que trabajen con matrices dispersas:

In [ ]:
from sklearn.metrics.pairwise import cosine_distances as dist_disp
# Se puede utilizar algunas de las métricas incluidas en 'sklearn.matrix.pairwise' 
# las cuales son: cosine_distances, euclidean_distances y manhattan_distances.

def calcular_similitud_disp(vec1, vec2):
  return 1/(1 + dist_disp(vec1, vec2)[0][0]) 

In [ ]:
def calcular_similitudes_disp(vectores, similar, similitud):
  cantVectores = vectores.shape[0]
  similitudes = [[] for _ in range(cantVectores)]
  for i in range(cantVectores):
    for j in range(i+1, cantVectores):
        dist = calcular_similitud_disp(vectores[i], vectores[j])
        heappush(similitudes[i], (dist, j))
        heappush(similitudes[j], (dist, i))
    maximo = max(similitudes[i])
    similitud[i] = maximo[0]
    similar[i] = maximo[1]

Luego, ejecutamos las mismas funciones que se utilizaron anteriormente para calcular los diccionarios similar y similitud
e imprimir los comentarios más similares:

In [ ]:
tComienzo = timer()
calcular_similitudes_disp(vectores[:1000], similar, similitud)
imprimir_similares(comentarios, similar, similitud)
tFinal = timer()
print("Demoró {} segundos.".format(round(tFinal - tComienzo, 3)))

Los resultados obtenidos son exactamente los mismos que los obtenidos en la implementación anterior. Se puede ver que al utilizar las funciones que contemplan  el manejo de matrices dispersas, la eficiencia computacional empeora considerablemente. Esto puede suceder ya que las funciones de distancia provistas por la librería Scikit Learn no son muy eficientes, incluso para el manejo de matrices dispersas. 

Analice los resultados obtenidos. ¿Que observa? ¿En qué opina que se podría mejorar?

Utilizando el parámetro 'max_df' en 0.50, pudimos ver que los mejores resultados se obtuvieron con oraciones cortas. Esto se puede explicar observando que las palabras son muy comunes en el tipo de comentarios que se está analizando (excelente, muy, bueno, buena, comida, etc.). En el caso de oraciones largas es muy probable que, aunque puedan contener estas palabras, el resto de las palabras no aparezcan comúnmente, por lo que aunque el valor de similitud sea alto los comentarios no son similares a la vista.

Otro aspecto a considerar es que es probable que los comentarios contengan faltas de ortografía o errores de tipeo. En nuestro, por lo indicado en la sección de Bag of Words, no se consideran estas palabras, lo que puede ser deseable (por ejemplo, puedo querer considerar la palabra 'comdia'). En estos casos, se podría utilizar una medida de distancia entre strings, como la distancia de Levenshtein o Jaro-Winkler, para mejorar los valores de similitud.

También existen implementaciones de las librerías Scikit Learn y Scipy que ya calculan las distancias entre cada par de filas de una matriz. Se podrían utilizar estas implementaciones en lugar de la implementada en calcular_similitudes para mejorar la eficiencia computacional.

## Bag of Words (con stemming)

Utilice el parámetro 'tokenizer' de la clase CountVectorizer y conjuntamente la clase SpanishStemmer del módulo nltk.stem.snowball. Repita los experimentos realizados en la parte anterior utilizando el stemmer. Realice las comparaciones que considere pertinentes.

In [ ]:
from nltk.stem.snowball import SpanishStemmer

spanish_stemmer = SpanishStemmer()

# Se define una nueva función que servira como nuevo parametro tokenizer del constructor CountVectorizer
def stem_tokenizer(text):
    tokenizer = transf.build_tokenizer()
    return [spanish_stemmer.stem(word) for word in tokenizer(text)]

In [ ]:
from copy import copy

# Se realiza una copia limpia de la instancia original de manera de 
# tener los mismos parametros iniciales de la seccion Bag of Word, 
# y a esta nueva instancia poder setearle el nuevo tokenizador utilizando el SpanishStemmer 
transf_with_stem = copy(transf).set_params(tokenizer=stem_tokenizer)
transf_with_stem.fit(comentarios)

In [ ]:
# Al igual que en las secciones anteriores imprimimos el diccionario y su tamaño

diccionario_with_stem = transf_with_stem.get_feature_names()
print(diccionario_with_stem)
print("El diccionario con stemmming tiene " + str(len(diccionario_with_stem)) + " palabras.")

In [ ]:
vectores_with_stem = transf_with_stem.transform(comentarios)

Al igual que en la parte anterior muestre los resultados para una porción del corpus y si lo desea incluya los resultados para una porción mayor. Despliegue el tiempo transcurrido.

In [ ]:
from timeit import default_timer as timer

similar = {}
similitud = {}

tComienzo = timer()
calcular_similitudes(vectores_with_stem.toarray()[:1000], similar, similitud)
imprimir_similares(comentarios, similar, similitud)
tFinal = timer()
print("Demoró {} segundos.".format(round(tFinal - tComienzo, 3)))

Justifique las decisiones tomadas y realice los comentarios que considere adecuados. Analice los resultados obtenidos. Compare los resultados con los obtenidos anteriormente. Comente los problemas que detecte en este enfoque.

Se toma como instancia base (`transf`) la obtenida en la sección *Bag of Word*, 
a fin de realizar una comparación acorde con la sección anterior (Similitud).

A la instancia original de *Bag of Word* se le aplica como nuevo parametro tokenizer, la funcion `stem_tokenizer`.
La misma calcula el *stem* para cada token obtenido de la instancia base, es decir, se convierte
el diccionario original a un nuevo diccionario con *stem*, 
tal que para cada token se aplico el algoritmo *SpanishStemmer*.

**Las observaciones realizadas son las siguientes:**

- Entrenar la transformación requiere más tiempo de ejecución, esto se debe a que se agrega un nuevo paso de procesamiento como tokenizer, para cada palabra obtenida del diccionario original (1818 en total), se aplica el algoritmo de *Stemming de Porter*, el cual conlleva al menos 5 pasos y más de 60 reglas.

- El diccionario con *stemming* se redujo en tamaño en aproximadamente un 24%, teniendo originalmente 1818 palabras, mientras que el diccionario con *stem* pasa a tener un total de 1388 palabras.

- La ejecución `calcular_similitudes` e `imprimir_similares` redujo su tiempo de 36.0 a 33.6 segundos en promedio para una porción de 1000 comentarios, que representa tan sólo el 2% de la totalidad. Es razónable pensar que esta brecha entre ambos tiempos, comience a ser aún mayor a medida que se incremente la porción del corpus utilizado.

- La impresión de los 50 comentarios más similares (y por tanto el calculo de similitud) dio distintos resultados.

        
**A partir de las observaciones realizadas creemos conveniente comentar algunas ventajas y desventajas a la hora de utilizar este método:**

*Stemming* es una fase de preprocesamiento comprendida a la hora de normalizar el texto, la cual consiste en reducir la palabra original, a su *stem*, mediante la aplicación de ciertas reglas ordenadas, las cuales van quitando afijos.
Muchas palabras son derivaciones del mismo *stem*, y se puede considerar que pertenecen al mismo concepto, es por esta razón que en algunas ocaciones es de interes agruparlas como un mismo concepto, como sucede en el problema actual de encontrar la similitud de comentarios.

El objetivo del *Stemming* es reducir las formas derivadas de una palabra a una forma base común para todas ellas. Por ejemplo: aplicar *stem* a: `abarca`, `abarcar`, `abarcado` da como resultado `abarc`. Notar que el *stem* `abarc` y en general cualquier *stem* no tiene porque pertenecer a un diccionario en español (no confundir con lema).

Es por esta razón que el tamaño del diccionario se redujo en tamaño en un 24%, ya que palabras similares, y en general con similar significado (es lo esperable), se agrupan en una forma base común. 
Reducir el tamaño del mismo es de suma importancia, ya que a menor cantidad de features, menor sera el procesamiento realizado por los algoritmos posteriores al pre-procesamiento, se pudo observar este comportamiento a la hora de ejecutar `calcular_similitudes` e `imprimir_similares`, que al procesar vectores con menor tamaño los mismos redujeron su tiempo de ejecución. 

Por lo tanto incrementamos el tiempo de entrenamiento una única vez, a costa de reducir el tiempo en los algoritmos futuros. Y lo que es más, también se redujo el tamaño de las estructuras de datos, por lo que las capacidades de almacenamiento necesarias son menores a la hora de manejar el modelo de *bag of words* con *stemming* en memoria o en su persistencia en disco.

Las desventajas de estos métodos es que para ciertas palabras, las reduce a un mismo concepto, cuando en realidad el concepto es distinto (falso positivo). Esto es lo que se conoce como `over-stemming` y a efectos de nuestro problema, esto afectaría en considerar comentarios como más similares de lo que realmente lo son. 

Otro error existente es el de `under-stemming`, el cual sucede cuando dos palabras con igual *stem* son reducidas a *stem* distintos (falso negativo). Nuevamente esto nos trae problemas dado que consideraríamos que ciertos comentarios no son similares cuando en realidad si lo son.

En general el algóritmo de Porter da buenos resultados, es extensible a cualquier lenguaje y aumenta la *recall* sin afectar la *precision*. (A Comparative Study of Stemming Algorithms, Ms. Anjali Ganesh Jivani)

In [ ]:
print(spanish_stemmer.stem("abarca"))
print(spanish_stemmer.stem("abarcar"))
print(spanish_stemmer.stem("abarcado"))

# Especificaciones técnicas:

    $ lshw
    
    ===>
    
          *-core
               description: Motherboard
               product: TP500LAB
               vendor: ASUSTeK COMPUTER INC.
               physical id: 0
               version: 1.0
               serial: BSN12345678901234567
               slot: MIDDLE
             *-firmware
                  description: BIOS
                  vendor: American Megatrends Inc.
                  physical id: 0
                  version: TP500LAB.204
                  date: 12/11/2014
                  size: 64KiB
                  capacity: 6400KiB
                  capabilities: pci upgrade shadowing cdboot bootselect socketedrom edd int13floppy1200 int13floppy720 int13floppy2880 int5printscreen int9keyboard int14serial int17printer acpi usb smartbattery biosbootspecification uefi
             *-cache:0
                  description: L1 cache
                  physical id: e
                  slot: L1 Cache
                  size: 32KiB
                  capacity: 32KiB
                  capabilities: synchronous internal write-back data
                  configuration: level=1
             *-cache:1
                  description: L1 cache
                  physical id: f
                  slot: L1 Cache
                  size: 32KiB
                  capacity: 32KiB
                  capabilities: synchronous internal write-back instruction
                  configuration: level=1
             *-cache:2
                  description: L2 cache
                  physical id: 10
                  slot: L2 Cache
                  size: 256KiB
                  capacity: 256KiB
                  capabilities: synchronous internal write-back unified
                  configuration: level=2
             *-cache:3
                  description: L3 cache
                  physical id: 11
                  slot: L3 Cache
                  size: 4MiB
                  capacity: 4MiB
                  capabilities: synchronous internal write-back unified
                  configuration: level=3
             *-cpu
                  description: CPU
                  product: Intel(R) Core(TM) i7-5500U CPU @ 2.40GHz
                  vendor: Intel Corp.
                  physical id: 12
                  bus info: cpu@0
                  version: Intel(R) Core(TM) i7-5500U CPU @ 2.40GHz
                  serial: NULL
                  slot: SOCKET 0
                  size: 2877MHz
                  capacity: 3GHz
                  width: 64 bits
                  clock: 100MHz
                  capabilities: x86-64 fpu fpu_exception wp vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp constant_tsc arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb invpcid_single pti retpoline intel_pt spec_ctrl tpr_shadow vnmi flexpriority ept vpid fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap xsaveopt dtherm ida arat pln pts cpufreq
                  configuration: cores=2 enabledcores=2 threads=4
             *-memory
                  description: System Memory
                  physical id: 14
                  slot: System board or motherboard
                  size: 8GiB
                *-bank:0
                     description: SODIMM DDR3 Synchronous 1600 MHz (0,6 ns)
                     vendor: Samsung
                     physical id: 0
                     serial: 00000000
                     slot: ChannelA-DIMM0
                     size: 4GiB
                     width: 64 bits
                     clock: 1600MHz (0.6ns)
                *-bank:1
                     description: SODIMM DDR3 Synchronous 1600 MHz (0,6 ns)
                     product: HMT451S6BFR8A-PB
                     vendor: Hynix/Hyundai
                     physical id: 1
                     serial: 00707194
                     slot: ChannelB-DIMM0
                     size: 4GiB
                     width: 64 bits
                     clock: 1600MHz (0.6ns)
